In [ ]:
import os
import openpyxl
from collections import defaultdict
import re
from shutil import copyfile

In [89]:
# 记录了合并id的表格，每列为一个摄像头，id用空格隔开，不能有任何符号、空行
mergeExcelPath = '/home/swjtu/reid/PaddleDetection/output/D1D2/merge.xlsx'
# 存放需要合并的所有图片
picPath = '/home/swjtu/reid/PaddleDetection/output/D1D2/D1D2'
# 汇总了所有需要合并的摄像头的单个excel
excelPath = '/home/swjtu/reid/PaddleDetection/output/D1D2/D1D2.xlsx'
# 需要合并的摄像头的id
cam_ids = [1,2]

# 结果输出目录
savePath = '/home/swjtu/reid/PaddleDetection/output/D1D2/merge_result/'

wb = openpyxl.load_workbook(mergeExcelPath)
sheet = wb['Sheet1']
merge_data = []
pids = []
#并查集
fa = [i for i in range(5000*len(cam_ids)+10)]

def merge(i,j):
    fa[find(i)] = find(j);
def find(x):
    if x == fa[x]: return x
    else:
        fa[x] = find(fa[x])
        return fa[x]
for root, dirs, files in os.walk(picPath):
    for img in files:
        pid, cam, seq, frame, vdate = re.match(r'(\d*)_c(\d*)s(\d*)_(\d*)_(\d*)\.jpg',img).groups()
        pid_ = cam_ids.index(int(cam))*5000+int(pid)
        pids.append(pid_)
for row in sheet.rows:
    row_data = []
    for i, cam_id in enumerate(cam_ids):
        ids = str(row[i].value).split(' ')
        if row[i].value is None or ids is None or len(ids)==0: continue
        for pid in ids:
            if pid != '': row_data.append(i*5000 + int(pid))
    merge_data.append(row_data)
for data in merge_data:
    for item in data:
        # 并查集 合并操作
        merge(item, data[0])
fa_dict = defaultdict(set)
for pid in pids: #range(len(fa)):
    fa_dict[find(pid)].add(pid)
merge_list = list(fa_dict.values())
merge_list = sorted(merge_list,key = lambda x:min(list(x))) #按合并集合里最早出现的id排序
relabel = dict()
for i, item in enumerate(merge_list):
    for pid in item:
        relabel[pid] = i+1

# 图片重新命名
if not os.path.exists(savePath):
    os.makedirs(savePath)
if not os.path.exists(os.path.join(savePath,'crop')):
    os.makedirs(os.path.join(savePath,'crop'))
for root, dirs, files in os.walk(picPath):
    for img in files:
        pid, cam, seq, frame, vdate = re.match(r'(\d*)_c(\d*)s(\d*)_(\d*)_(\d*)\.jpg',img).groups()
        pid_ = cam_ids.index(int(cam))*5000+int(pid)
        new_label = relabel[pid_] if pid_ in relabel else int(pid)
        new_file = "%04d_c%ss%s_%s_%s.jpg" % (new_label, cam, seq, frame, vdate)
        # print(img,new_file)
        copyfile(os.path.join(root,img), os.path.join(os.path.join(savePath,'crop'),new_file))
        
# excel改id
xlsx = []
wb2 = openpyxl.load_workbook(excelPath)
ws2 = wb2['Sheet']
for row in ws2.rows:
    pid, in_time, out_time, in_frame, out_frame, cam_id, seq = [c.value for c in row]
    if pid == '人员ID': continue
    pid_ = cam_ids.index(int(cam))*5000+int(pid)
    new_label = relabel[pid_] if pid_ in relabel else int(pid)
    xlsx.append([new_label, in_time, out_time, in_frame, out_frame, cam_id, seq])
xlsx = sorted(xlsx, key = lambda x:x[0])
new_wb = openpyxl.Workbook()
new_ws = new_wb['Sheet']
new_ws.append(['人员ID', '进入时间', '离开时间', '进入帧号', '离开帧号', '摄像头ID', '视频片段序号'])
for i in xlsx:
    new_ws.append(i)
new_wb.save(os.path.join(savePath, 'test.xlsx'))